Stage 2:  Data preprocessing stage to produce structured data in csv format also stored in Cloud Object Storage. As columns in the csv file we suggest date, geographic location, url, and sentiment analysis.

In [ ]:
import lithops

In [ ]:
from transformers import pipeline
from custom_snscrape.twitter import Tweet

def map_preprocess(obj) -> dict:

    data_body_str = obj.data_stream.read().decode('utf-8')    
    tweet = Tweet.from_json(data_body_str)

    if tweet.lang not in ['es', 'en', 'it', 'fr', 'de', 'nl']:
        return

    classifier = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")
    sentiment = classifier(tweet.content)

    # TODO: Perhaps cast it all to string so we don't have to import the library inside reduce_to_csv
    return { 'id': tweet.id,
             'user': tweet.user.username,
             'content': tweet.content.replace('\n', ' '), # replace newlines so they don't break the csv
             'lang': tweet.lang,
             'coordinates': tweet.coordinates,
             'retweetCount': tweet.retweetCount,
             'likeCount': tweet.likeCount,
             'quoteCount': tweet.quoteCount,
             'replyCount': tweet.replyCount,
             'mentionedUsers': tweet.mentionedUsers,
             'outlinks': tweet.outlinks,
             'date': tweet.date.isoformat(),
             'sentiment': sentiment[0]['label'] }


In [ ]:
import csv
from io import StringIO


def reduce_to_csv(results, storage):
    
    # BUG: Because we are passing dataclasses from a pickled module inside map() function as its results, we must
    # import something from the library that contains the dataclasses or lithops won't find it.
    # If you don't believe me, remove this line :)
    from custom_snscrape.twitter import Tweet
    #############################################################################################################

    column_names = results[0].keys()
    csv_buffer = StringIO()

    with StringIO() as csv_buffer:
        dict_writer = csv.DictWriter(csv_buffer, column_names)
        dict_writer.writeheader()
        dict_writer.writerows(results)

        storage.put_object(bucket=storage.bucket,
                        key='test.csv',
                        body=csv_buffer.getvalue())

    return "reduced result to cos://{}/test.csv output and stored it".format(storage.bucket)

In [ ]:
hashtags = []

# Retrieve hashtags from Stage 1 notebook
%store -r hashtags

if len(hashtags) == 0:
    print('You have not defined any hashtags yet (use Stage 1 notebook or the \'hashtags\' variable directly)')

# Prepend bucket name and append / to the hashtags
bucket_name = lithops.Storage().bucket
data_location = [bucket_name+'/'+htag+'/' for htag in hashtags]

In [ ]:
with lithops.FunctionExecutor(runtime='gilarasa/lithops-cloudbutton-challenge-py3.9:0.5', runtime_memory=2048) as fexec:
  # , log_level='debug'

  fexec.map_reduce(map_preprocess, data_location, reduce_to_csv)
  print(fexec.get_result())